In [28]:
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T


In [29]:
class SignDigitsDataset(Dataset):
    def __init__(self, base_path, img_size, is_train=True, custom_transform=None):
        self.base_path = base_path
        self.img_size = img_size
        self.is_train = is_train
        self.transform = custom_transform if custom_transform else self.default_transform()
        self.img_list = []
        self.label_list = []
        self.prepare_dataset()

    def default_transform(self):
        return T.Compose([
            T.Resize(self.img_size),
            T.ToTensor(),
            T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])

    def prepare_dataset(self):
        for digit in range(10):
            digit_path = os.path.join(self.base_path, str(digit))
            if not os.path.isdir(digit_path):
                continue
            files = [f for f in os.listdir(digit_path) if f.endswith('.JPG')]
            cutoff = int(0.7 * len(files))

            chosen_files = files[:cutoff] if self.is_train else files[cutoff:]

            for f_name in chosen_files:
                full_path = os.path.join(digit_path, f_name)
                self.img_list.append(full_path)
                self.label_list.append(digit)

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        image = Image.open(self.img_list[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.label_list[idx]


In [30]:
batch = 64
img_dim = (64, 64)
data_root = r'C:\Users\Administrator IZ\Desktop\Sign-Language-Digits-Dataset\Dataset'

train_data = SignDigitsDataset(base_path=data_root, img_size=img_dim, is_train=True)
test_data = SignDigitsDataset(base_path=data_root, img_size=img_dim, is_train=False)

train_loader = DataLoader(dataset=train_data, batch_size=batch, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch, shuffle=False)

print(img_dim)
print(img_dim)


(64, 64)
(64, 64)


In [31]:
class DigitClassifier(nn.Module):
    def __init__(self, input_dim, output_classes):
        super(DigitClassifier, self).__init__()
        self.layer1 = nn.Linear(input_dim, 256)
        self.layer2 = nn.Linear(256, 64)
        self.output = nn.Linear(64, output_classes)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.output(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_dim = 64 * 64 * 3
classes = 10
lr = 0.001
epochs = 6

model = DigitClassifier(input_dim, classes).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for ep in range(epochs):
    print(f"Epoch [{ep+1}]")
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = inputs.view(inputs.size(0), -1)

        predictions = model(inputs)
        loss = loss_fn(predictions, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


Epoch [1]
Epoch [2]
Epoch [3]
Epoch [4]
Epoch [5]
Epoch [6]


In [32]:
def evaluate(loader, model):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.view(inputs.size(0), -1)

            outputs = model(inputs)
            _, predicted = outputs.max(dim=1)
            correct += (predicted == labels).sum()
            total += labels.size(0)
            break  # Check one batch only

    accuracy = 100 * correct.item() / total
    print(f"Accuracy: {correct}/{total} ({accuracy:.2f}%)")

print("Evaluating on Test Set:")
evaluate(test_loader, model)

print("Evaluating on Train Set:")
evaluate(train_loader, model)


Evaluating on Test Set:
Accuracy: 48/64 (75.00%)
Evaluating on Train Set:
Accuracy: 55/64 (85.94%)
